In [ ]:
## This file extracts data from the hdf files that contain the predictions coming from Gauthams OpenFace
## It gets the raw Happy/Not Happy predictions per frame, per session

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
from dask import dataframe as dd
import util.label_utils as util
from datetime import time
from multiprocessing import Pool

In [ ]:
for sess in [2,4,5,7,8,10,11]:
    link='/home/emil/data/hdf_data/split_by_video/cb46fd46_'+str(sess)+'*.hdf'
    rdd=dd.read_hdf(link,'/data')
    #@TODO add realtime once available
    imp_columns=['patient','success','frame','timestamp','annotated','confidence','session','vid','Happy_predicted','datetime','realtime']
    df=rdd[imp_columns].compute()

    #if success==False (i.e. could not detect AU), set happy pred to N/A
    df.loc[df.success==0, 'Happy_predicted'] = float('NaN')

    #sort by datetime and reset the indices
    df.sort_values(by=['realtime'],inplace=True)
    df.reset_index(drop=True,inplace=True)

    # def is_df_sorted(df, colname):
    #     return pd.Index(df[colname]).is_monotonic

    # is_df_sorted(df,'realtime')
    link_save='/home/emil/data/hdf_data/cb46fd46_'+str(sess)+'_imp_columns.hdf'
    df.to_hdf(link_save, key='df')

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

In [ ]:
mypath ='/home/emil/data/hdf_data/'

In [ ]:
sessions = [f for f in listdir(mypath) if isfile(join(mypath, f)) and 'imp_columns' in f]

In [ ]:
for idx,s in enumerate(sessions):
    curr_df = pd.read_hdf(join(mypath,s))
    if idx == 0:
        df = curr_df
    else:
        df = df.append(curr_df)

In [ ]:
#df = df.sort_values(by = ['datetime'])
#df[df['session']=='11']
df.drop('realtime', axis=1, inplace=True)

In [ ]:
t = pd.read_csv('/nas/ecog_project/derived/processed_ecog/cb46fd46/full_day_ecog/vid_start_end_merge.csv')

#
I want to order my hdf files not by session but by day. How?
1. For each line of above df, create the videofile name.
2. Find the corresponding time and day in the start_end_vid file
3. Add to columns

In [ ]:
def fill_row(index,row):
    #create the filename from line
    #row = df_old.iloc[index]
    pat = row['patient']
    sess = row['session']
    vid = row['vid']
    fname = '_'.join([pat,sess,vid])
    fname = '.'.join([fname,'avi'])
    #find the file in the videotime infostuff
    line = t[t['filename']==fname]
    #crate time object
    vid_time = time(line['hour'].iloc[0],line['minute'].iloc[0],line['second'].iloc[0],line['microsecond'].iloc[0])
    #add column wih the important info
    return [*row,vid_time,line['merge_day'].iloc[0]]

In [ ]:
all_rows = df.iterrows()

In [ ]:
pool = Pool(8)
yass = pool.starmap(fill_row,all_rows)
new_df = pd.DataFrame( yass, columns = [*df.columns,'steve_time','merge_day'])


In [ ]:
for day in pd.unique(new_df['merge_day']):
    curr_df = new_df[new_df['merge_day']==day]
    sorted_curr_df = curr_df.sort_values(by=['session','vid','frame']).reset_index(drop=True)
    sorted_curr_df.to_hdf('/home/emil/data/hdf_data/by_day/cb46fd46_day_'+str(day)+'.hdf',key='df')

In [ ]:
test = pd.read_hdf('/home/emil/data/hdf_data/by_day/cb46fd46_day_1.hdf')
